## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
0,0,Ponta Do Sol,32.6667,-17.1000,65.03,57,93,11.48,overcast clouds,PT,2022-01-22 03:13:37
1,1,Minas,-34.3759,-55.2377,73.87,79,95,11.18,very heavy rain,UY,2022-01-22 03:13:37
2,2,Dikson,73.5069,80.5464,14.29,96,100,6.13,overcast clouds,RU,2022-01-22 03:13:37
3,3,Talnakh,69.4865,88.3972,11.14,91,100,11.23,overcast clouds,RU,2022-01-22 03:13:38
4,4,Victoria,22.2855,114.1577,66.18,93,100,3.00,overcast clouds,HK,2022-01-22 03:13:38


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country,Date
6,6,Atuona,-9.8000,-139.0333,78.80,78,16,19.01,light rain,PF,2022-01-22 03:13:39
7,7,Mitsamiouli,-11.3847,43.2844,80.94,71,1,9.28,light rain,KM,2022-01-22 03:13:39
9,9,Rikitea,-23.1203,-134.9692,78.62,76,47,11.99,scattered clouds,PF,2022-01-22 03:13:40
11,11,Hithadhoo,-0.6000,73.0833,81.37,72,100,9.40,overcast clouds,MV,2022-01-22 03:13:40
12,12,Kapaa,22.0752,-159.3190,79.12,70,0,1.01,clear sky,US,2022-01-22 03:12:22
13,13,Pitimbu,-7.4706,-34.8086,77.59,77,24,8.99,few clouds,BR,2022-01-22 03:13:41
16,16,Hilo,19.7297,-155.0900,75.79,81,1,1.01,clear sky,US,2022-01-22 03:10:59
17,17,Pochutla,15.7432,-96.4661,76.82,83,14,3.47,few clouds,MX,2022-01-22 03:13:41
20,20,Flinders,-34.5833,150.8552,77.81,63,88,8.79,overcast clouds,AU,2022-01-22 03:13:42
21,21,Luderitz,-26.6481,15.1594,79.75,63,100,10.38,overcast clouds,NaN,2022-01-22 03:13:42


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                155
City                   155
Lat                    155
Lng                    155
Max Temp               155
Humidity               155
Cloudiness             155
Wind Speed             155
Weather Description    155
Country                154
Date                   155
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                154
City                   154
Lat                    154
Lng                    154
Max Temp               154
Humidity               154
Cloudiness             154
Wind Speed             154
Weather Description    154
Country                154
Date                   154
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
6,Atuona,PF,78.80,light rain,-9.8000,-139.0333,
7,Mitsamiouli,KM,80.94,light rain,-11.3847,43.2844,
9,Rikitea,PF,78.62,scattered clouds,-23.1203,-134.9692,
11,Hithadhoo,MV,81.37,overcast clouds,-0.6000,73.0833,
12,Kapaa,US,79.12,clear sky,22.0752,-159.3190,
13,Pitimbu,BR,77.59,few clouds,-7.4706,-34.8086,
16,Hilo,US,75.79,clear sky,19.7297,-155.0900,
17,Pochutla,MX,76.82,few clouds,15.7432,-96.4661,
20,Flinders,AU,77.81,overcast clouds,-34.5833,150.8552,
27,Sambava,MG,75.67,light rain,-14.2667,50.1667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
  
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
6,Atuona,PF,78.80,light rain,-9.8000,-139.0333,Villa Enata
7,Mitsamiouli,KM,80.94,light rain,-11.3847,43.2844,Foyer ADM
9,Rikitea,PF,78.62,scattered clouds,-23.1203,-134.9692,People ThankYou
11,Hithadhoo,MV,81.37,overcast clouds,-0.6000,73.0833,Scoop Guest House
12,Kapaa,US,79.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Pitimbu,BR,77.59,few clouds,-7.4706,-34.8086,Reserva do Abiaí
16,Hilo,US,75.79,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Pochutla,MX,76.82,few clouds,15.7432,-96.4661,Hotel Posada San Jose
20,Flinders,AU,77.81,overcast clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
27,Sambava,MG,75.67,light rain,-14.2667,50.1667,Melrose


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))